In [15]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython import display

SHOW_GIFS = False

In [2]:
import numpy as np
import matplotlib.pylab as plt
import functions as fn
import execution as pso
import strategies
import plotting as plot

In [14]:
with fn.get_functions(dimension=[2], function_indices=[5]) as suite:
    for function in suite:  # type: fn.cocoex.Problem
        populations, values = pso.execute_multiple(
            function,
            strategies.random_walk,
            repeats=200,
            show_progress=True
        )
        
        #plotting.plot_population(function, populations[-5])
        #plotting.plot_population(function, populations[-4])
        #plotting.plot_population(function, populations[-3])
        #plotting.plot_population(function, populations[-2])
        #plotting.plot_population(function, populations[-1])

        #plotting.plot_movement_of_individual(function, populations[:,5,:])

        if SHOW_GIFS:
            gif = plot.animate_movement(function, populations, figsize=(12,8))
            display.Image(data=gif, format='png')

100% (200 of 200) |######################| Elapsed Time: 0:00:20 Time:  0:00:20


In [ ]:
plt.figure(figsize=(12,8))

for val in values:
    plot.plot_graph(val, plot.plotfn_median(), alpha=0.2)
plot.plot_aggregated(values+1, plot.plotfn_median(), plot.aggfn_mean(), label="mean", c='r', alpha=0.3)
plot.plot_aggregated(values+1, plot.plotfn_median(), plot.aggfn_median(), label="median", c='b', alpha=0.3)

plt.legend()
plt.show()